In [1]:
import numpy as np
import os
import cv2 as cv
from typing import Iterable
import matplotlib.pyplot as plt
import glob
import time

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torchvision.transforms as T
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils import data
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from models import MixtureOfExpertsEncoder, Traditional2dSegmenter
from models.utils.util_classes import SplitTensor, AddInQuadrature, DepthSum, ConvWH, ConvDW, ConvDH

dev = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(dev)
print(device)

cpu


In [2]:
class FftLayer(nn.Module):
    def __init__(self, kernel_size, stride, padding, num_orders=4, inverse=False):
        super().__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.num_orders = num_orders

In [3]:
root_directory = '/home/shiva/data/ImageSegmentationFloodZones/'
image_directory = os.path.join(root_directory, 'Image')
mask_directory = os.path.join(root_directory, 'Mask')
image_paths = glob.glob(image_directory + '/*.jpg')
mask_paths = glob.glob(mask_directory + '/*.png')
print(image_paths)
print(mask_paths)

['/home/shiva/data/ImageSegmentationFloodZones/Image/1003.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/3061.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/1068.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/3006.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/3032.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/3077.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/3016.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/1075.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/3079.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/28.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/3098.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/1067.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/3000.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/3010.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/3025.jpg', '/home/shiva/data/ImageSegmentationFloodZones/Image/1020

In [4]:
im_transform = T.Compose([
    T.ToTensor(),
    T.Resize((256, 256)),
])

msk_transform = T.Compose([
    T.ToTensor(),
    T.Resize((53, 53)),
])

def to_image(inp):
    if inp.ndim == 3:
        out = inp.numpy()
        out = np.transpose(out, (1, 2, 0))
    else:
        out = inp.numpy()
        out = np.transpose(out, (0, 2, 3, 1))
    return out

In [5]:
class FloodZoneDataset(data.Dataset):
    def __init__(self, root_directory, im_transform=im_transform, msk_transform=msk_transform):
        self.image_directory = os.path.join(root_directory, 'Image')
        self.mask_directory = os.path.join(root_directory, 'Mask')
        self.im_transform = im_transform
        self.msk_transform = msk_transform
        self.image_paths = sorted(glob.glob(os.path.join(self.image_directory, '*.jpg')))
        self.mask_paths = sorted(glob.glob(os.path.join(self.mask_directory, '*.png')))
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]
        image = cv.imread(image_path)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
        image = self.im_transform(image)
        mask = self.msk_transform(mask)
        return image, 1.0 - mask
    
    def __len__(self):
        return len(self.image_paths)

In [6]:
dataset = FloodZoneDataset(root_directory)
dataloader = data.DataLoader(dataset, batch_size=4, shuffle=True, drop_last=True)

X, y = next(iter(dataloader))
print(X.shape, y.shape)

torch.Size([4, 3, 256, 256]) torch.Size([4, 1, 53, 53])


In [7]:
class MixtureOfExpertsDecoder(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        pass

In [8]:
test_encoder = MixtureOfExpertsEncoder.MixtureOfExpertsSegmentationEncoder()
ctrl_encoder = Traditional2dSegmenter.ControlSegmentationModel()
start = time.time()
out = ctrl_encoder(X)
print(time.time() - start)
print(out.shape)
print(out.min(), out.max())

start = time.time()
out = test_encoder(X)
print(time.time() - start)
print(out.shape)
print(out.min(), out.max())

2.4319140911102295
torch.Size([4, 96, 53, 53])
tensor(0., grad_fn=<MinBackward1>) tensor(1., grad_fn=<MaxBackward1>)
2.305164337158203
torch.Size([4, 96, 53, 53])
tensor(0., grad_fn=<MinBackward1>) tensor(1., grad_fn=<MaxBackward1>)
